<p align="center" width="100%">
    <img width="40%" src="customer_support_icon.JPG"> 
</p>

A retail company is on a transformative journey, aiming to elevate their customer services through cutting-edge advancements in Speech Recognition and Natural Language Processing (NLP). As the machine learning engineer for this initiative, you are tasked with developing functionalities that not only convert customer support audio calls into text but also explore methodologies to extract insights from transcribed texts.

In this dynamic project, we leverage the power of `SpeechRecognition`, `Pydub`, and `spaCy` – three open-source packages that form the backbone of your solution. Your objectives are:
  - Transcribe a sample customer audio call, stored at `sample_customer_call.wav`, to showcase the power of open-source speech recognition technology.
  - Analyze sentiment, identify common named entities, and enhance user experience by searching for the most similar customer calls based on a given query from a subset of their pre-transcribed call data, stored at `customer_call_transcriptions.csv`.

This project is an opportunity to unlock the potential of machine learning to revolutionize customer support. Let's delve into the interplay between technology and service excellence.

In [1]:
!pip install SpeechRecognition
!pip install pydub
!pip install spacy
!python3 -m spacy download en_core_web_sm

Defaulting to user installation because normal site-packages is not writeable
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 32.8/32.8 MB 147.4 MB/s eta 0:00:0000:01

[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python3 -m pip install --upgrade pip
Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python3 -m pip install --upgrade pip
Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python3 -m pip install --upgrade pip
Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 151.2 MB/s eta 0:00:00

[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python3 -m pip install --upgrade pip
✔ Download and installation succes

In [2]:
import pandas as pd

import nltk
nltk.download('vader_lexicon')
from nltk.sentiment.vader import SentimentIntensityAnalyzer

import speech_recognition as sr
from pydub import AudioSegment

import spacy

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /home/repl/nltk_data...


In [3]:
# Task 1 - Speech to Text: แปลงไฟล์เสียงจาก audio call (sample_customer_call.wav) เป็นข้อความ และเก็บผลลัพธ์ใน transcribed_text

# สร้าง object recognizer สำหรับแปลงเสียงเป็นข้อความ
recognizer = sr.Recognizer()

# แปลงไฟล์เสียงเป็นข้อมูลเสียง
transcribe_audio_file = sr.AudioFile("sample_customer_call.wav")
with transcribe_audio_file as source:
    transcribe_audio = recognizer.record(source)

# แปลงข้อมูลเสียงเป็นข้อความโดยใช้ Google Speech Recognition API
transcribed_text = recognizer.recognize_google(transcribe_audio)

# แสดงข้อความที่แปลงจากเสียง
print("ข้อความที่แปลงจากเสียง: ", transcribed_text)

# Task 1 - Speech to Text: เก็บข้อมูลสถิติของไฟล์เสียง เช่น จำนวนช่องเสียง (channels), ความกว้างของตัวอย่าง (sample width), และอัตราเฟรม (frame rate)

# ตรวจสอบจำนวนช่องเสียง (channels) และอัตราเฟรม (frame rate) ของไฟล์เสียง
audio_segment = AudioSegment.from_file("sample_customer_call.wav")
number_channels = audio_segment.channels
frame_rate = audio_segment.frame_rate

# แสดงจำนวนช่องเสียงและอัตราเฟรมของไฟล์เสียง
print("จำนวนช่องเสียง: ", number_channels)
print("อัตราเฟรม: ", frame_rate)


ข้อความที่แปลงจากเสียง:  hello I'm experiencing an issue with your product I'd like to speak to someone about a replacement
จำนวนช่องเสียง:  1
อัตราเฟรม:  44100


In [4]:
# Task 2 - Sentiment Analysis: วิเคราะห์ความคิดเห็นจากข้อความในไฟล์ customer_call_transcriptions.csv โดย Vader จากไลบรารี nltk และมีการเพิ่มคอลัมน์ใหม่ใน DataFrame สำหรับเก็บผลลัพธ์ความคิดเห็น

df = pd.read_csv("customer_call_transcriptions.csv")

sid = SentimentIntensityAnalyzer()

# ฟังก์ชันวิเคราะห์ความคิดเห็น
def find_sentiment(text):
    scores = sid.polarity_scores(text)
    compound_score = scores['compound']

    if compound_score >= 0.05:
        return 'positive'
    elif compound_score <= -0.05:
        return 'negative'
    else:
        return 'neutral'

df['sentiment_predicted'] = df.apply(lambda row: find_sentiment(row["text"]), axis = 1)

# Task 2 - Sentiment Analysis: คำนวณ True Positive
true_positive = len(df.loc[(df['sentiment_predicted'] == df['sentiment_label']) &
                (df['sentiment_label'] == 'positive')])

print("True positives: ", true_positive)

True positives:  2


In [5]:
# Task 3 - Named Entity Recognition: ค้นหา Named Entities จากข้อความในแต่ละแถวของ DataFrame และจัดเก็บผลลัพธ์ไว้ในคอลัมน์ใหม่

nlp = spacy.load("en_core_web_sm")

# ฟังก์ชันสำหรับดึง Named Entities (NER) จากข้อความ
def extract_entities(text):
    doc = nlp(text)
    entities = [ent.text for ent in doc.ents]
    return entities

# ใช้ NER กับคอลัมน์ข้อความทั้งหมดใน DataFrame
df['named_entities'] = df['text'].apply(extract_entities)

# ทำให้รายการ named entities เป็นลิสต์เดียว (flatten)
all_entities = [ent for entities in df['named_entities'] for ent in entities]

# สร้าง DataFrame ที่นับจำนวน named entities
entities_df = pd.DataFrame(all_entities, columns=['entity'])
entities_counts = entities_df['entity'].value_counts().reset_index()
entities_counts.columns = ['entity', 'count']

# ดึง Named Entity ที่พบมากที่สุด
most_freq_ent = entities_counts["entity"].iloc[0]
print("Most frequent entity: ", most_freq_ent)


Most frequent entity:  yesterday


In [6]:
# Task 4 - Find most similar text: ค้นหาลิสต์ของข้อความจากลูกค้าที่ร้องเรียนเกี่ยวกับ "wrong package delivery" โดยคำนวณคะแนนความเหมือน (similarity score) ของแต่ละข้อความใน DataFrame เทียบกับข้อความต้นแบบ "wrong package delivery" โดยใช้spaCy

nlp = spacy.load("en_core_web_sm")

# ประมวลผลข้อความในคอลัมน์ text
df['processed_text'] = df['text'].apply(lambda text: nlp(text))

# ข้อความที่ต้องการค้นหา (Input Query)
input_query = "wrong package delivery"
processed_query = nlp(input_query)

# คำนวณคะแนนความเหมือน (Similarity Score) และจัดเรียง DataFrame ตามคะแนน
df['similarity'] = df['processed_text'].apply(lambda text: processed_query.similarity(text))
df = df.sort_values(by='similarity', ascending=False)

# ค้นหาข้อความที่เหมือนที่สุด
most_similar_text = df["text"].iloc[0]
print("Most similar text: ", most_similar_text)


Most similar text:  wrong package delivered
